<a href="https://colab.research.google.com/github/tyoc213/fastai_xla_extensions/blob/explorations1/explore_nbs/Basic_lenet_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install fastai2 from github

In [1]:
!pip install git+https://github.com/fastai/fastcore
!pip install git+https://github.com/fastai/fastai2

  Cloning https://github.com/fastai/fastcore to /tmp/pip-req-build-4gnvymab
  Running command git clone -q https://github.com/fastai/fastcore /tmp/pip-req-build-4gnvymab
  Created wheel for fastcore: filename=fastcore-0.1.18-cp36-none-any.whl size=28891 sha256=5bff4a6579e31bbd71758d287696fe4c10fece669a2e79a65ea35d862dd64e81
  Stored in directory: /tmp/pip-ephem-wheel-cache-1knpnvnf/wheels/8a/2a/23/bc50c8f5e28776b44ac837a01fcfa675724565d4813d8e51c7
Successfully built fastcore
  Cloning https://github.com/fastai/fastai2 to /tmp/pip-req-build-_tyyk9kf
  Running command git clone -q https://github.com/fastai/fastai2 /tmp/pip-req-build-_tyyk9kf
  Created wheel for fastai2: filename=fastai2-0.0.18-cp36-none-any.whl size=193924 sha256=b013a866f8ffd9d0103e9825f61d7da290558552273f19f8242c982762d33f44
  Stored in directory: /tmp/pip-ephem-wheel-cache-wqxey6s_/wheels/a1/59/9a/50335b36924b827e29d5f40b41fc3a008cc1f30dd80e560dfd
Successfully built fastai2


In [2]:
from fastai2.vision.all import *

In [12]:
path = untar_data(URLs.MNIST_SAMPLE)
path.ls()[2].ls()

(#2) [Path('/root/.fastai/data/mnist_sample/train/7'),Path('/root/.fastai/data/mnist_sample/train/3')]

In [13]:
def get_my_labels(fname):
    return int(fname.parent.name[0])

dblock = DataBlock(
    splitter = RandomSplitter(),
    #item_tfms = Resize(128),
    blocks = (ImageBlock, CategoryBlock),
    get_items = get_image_files,
    get_y = get_my_labels
)


In [14]:
dls = dblock.dataloaders(path)
dls.vocab

(#2) [3,7]

# Lenet with convs and F.max_pool2d

In [31]:
class MyLenet(nn.Module):
    def __init__(self):
        super(MyLenet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 3)
        self.conv2 = nn.Conv2d(6,16,3)
        self.hiden4 = nn.Linear(400, 2) # 2 outputs instead of 10
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = self.hiden4(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

lenet = MyLenet()
learn = Learner(dls, lenet, metrics=[error_rate, accuracy])

learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,error_rate,accuracy,time
0,0.097397,0.059037,0.020097,0.979903,00:14


In [20]:
learn.validate()

(#3) [0.06825366616249084,0.02460152469575405,0.9753984808921814]

# Lenet with layers

In [30]:
class Lenet2(nn.Module):
    def __init__(self):
        super(Lenet2, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.fc1 = nn.Linear(400, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2) # Only 2 outputs instead of 10
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features



lenet2 = Lenet2()
learn2 = Learner(dls, lenet2, metrics=[error_rate, accuracy])

learn2.fit_one_cycle(1)

epoch,train_loss,valid_loss,error_rate,accuracy,time
0,0.050592,0.030985,0.013167,0.986833,00:14


In [25]:
learn.validate()

(#3) [0.0628962516784668,0.02460152469575405,0.9753984808921814]